# https://medium.com/@thechrisyoon/deriving-policy-gradients-and-implementing-reinforce-f887949bd63

In [2]:
import sys
import torch  
import gym
import numpy as np  
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt

# Constants
GAMMA = 0.9

# policy network

class PolicyNetwork(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, learning_rate=3e-4):
        super(PolicyNetwork, self).__init__()

        self.num_actions = num_actions
        self.linear1 = nn.Linear(num_inputs, hidden_size)
        self.linear2 = nn.Linear(hidden_size, num_actions)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.softmax(self.linear2(x), dim=1)
        return x 
    
    def get_action(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0)
        probs = self.forward(Variable(state))
        highest_prob_action = np.random.choice(self.num_actions, p=np.squeeze(probs.detach().numpy()))
        log_prob = torch.log(probs.squeeze(0)[highest_prob_action])
        return highest_prob_action, log_prob


In [3]:
def update_policy(policy_network, rewards, log_probs):
    discounted_rewards = []

    for t in range(len(rewards)):
        Gt = 0 
        pw = 0
        for r in rewards[t:]:
            Gt = Gt + GAMMA**pw * r
            pw = pw + 1
        discounted_rewards.append(Gt)
        
    discounted_rewards = torch.tensor(discounted_rewards)
    discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-9) # normalize discounted rewards

    policy_gradient = []
    for log_prob, Gt in zip(log_probs, discounted_rewards):
        policy_gradient.append(-log_prob * Gt)
    
    policy_network.optimizer.zero_grad()
    policy_gradient = torch.stack(policy_gradient).sum()
    policy_gradient.backward()
    policy_network.optimizer.step()

In [4]:
def main():
    env = gym.make('CartPole-v0')
    policy_net = PolicyNetwork(env.observation_space.shape[0], env.action_space.n, 128)
    
    max_episode_num = 5000
    max_steps = 10000
    numsteps = []
    avg_numsteps = []
    all_rewards = []

    for episode in range(max_episode_num):
        state = env.reset()
        log_probs = []
        rewards = []

        for steps in range(max_steps):
            env.render()
            action, log_prob = policy_net.get_action(state)
            new_state, reward, done, _ = env.step(action)
            log_probs.append(log_prob)
            rewards.append(reward)

            if done:
                update_policy(policy_net, rewards, log_probs)
                numsteps.append(steps)
                avg_numsteps.append(np.mean(numsteps[-10:]))
                all_rewards.append(np.sum(rewards))
                if episode % 1 == 0:
                    sys.stdout.write("episode: {}, total reward: {}, average_reward: {}, length: {}\n".format(episode, np.round(np.sum(rewards), decimals = 3),  np.round(np.mean(all_rewards[-10:]), decimals = 3), steps))
                break
            
            state = new_state
        
    plt.plot(numsteps)
    plt.plot(avg_numsteps)
    plt.xlabel('Episode')
    plt.show()

In [ ]:
main()

episode: 0, total reward: 15.0, average_reward: 15.0, length: 14
episode: 1, total reward: 11.0, average_reward: 13.0, length: 10
episode: 2, total reward: 35.0, average_reward: 20.333, length: 34
episode: 3, total reward: 11.0, average_reward: 18.0, length: 10
episode: 4, total reward: 21.0, average_reward: 18.6, length: 20
episode: 5, total reward: 22.0, average_reward: 19.167, length: 21
episode: 6, total reward: 11.0, average_reward: 18.0, length: 10
episode: 7, total reward: 24.0, average_reward: 18.75, length: 23
episode: 8, total reward: 19.0, average_reward: 18.778, length: 18
episode: 9, total reward: 17.0, average_reward: 18.6, length: 16
episode: 10, total reward: 43.0, average_reward: 21.4, length: 42
episode: 11, total reward: 17.0, average_reward: 22.0, length: 16
episode: 12, total reward: 17.0, average_reward: 20.2, length: 16
episode: 13, total reward: 16.0, average_reward: 20.7, length: 15
episode: 14, total reward: 10.0, average_reward: 19.6, length: 9
episode: 15, t

episode: 125, total reward: 21.0, average_reward: 20.2, length: 20
episode: 126, total reward: 12.0, average_reward: 20.3, length: 11
episode: 127, total reward: 25.0, average_reward: 19.1, length: 24
episode: 128, total reward: 34.0, average_reward: 21.1, length: 33
episode: 129, total reward: 14.0, average_reward: 21.3, length: 13
episode: 130, total reward: 37.0, average_reward: 23.2, length: 36
episode: 131, total reward: 18.0, average_reward: 23.1, length: 17
episode: 132, total reward: 14.0, average_reward: 21.9, length: 13
episode: 133, total reward: 40.0, average_reward: 22.5, length: 39
episode: 134, total reward: 19.0, average_reward: 23.4, length: 18
episode: 135, total reward: 52.0, average_reward: 26.5, length: 51
episode: 136, total reward: 41.0, average_reward: 29.4, length: 40
episode: 137, total reward: 14.0, average_reward: 28.3, length: 13
episode: 138, total reward: 43.0, average_reward: 29.2, length: 42
episode: 139, total reward: 29.0, average_reward: 30.7, length

episode: 248, total reward: 16.0, average_reward: 26.7, length: 15
episode: 249, total reward: 34.0, average_reward: 27.4, length: 33
episode: 250, total reward: 83.0, average_reward: 30.6, length: 82
episode: 251, total reward: 12.0, average_reward: 30.6, length: 11
episode: 252, total reward: 19.0, average_reward: 31.3, length: 18
episode: 253, total reward: 48.0, average_reward: 35.0, length: 47
episode: 254, total reward: 28.0, average_reward: 29.3, length: 27
episode: 255, total reward: 26.0, average_reward: 29.8, length: 25
episode: 256, total reward: 36.0, average_reward: 32.2, length: 35
episode: 257, total reward: 34.0, average_reward: 33.6, length: 33
episode: 258, total reward: 28.0, average_reward: 34.8, length: 27
episode: 259, total reward: 59.0, average_reward: 37.3, length: 58
episode: 260, total reward: 33.0, average_reward: 32.3, length: 32
episode: 261, total reward: 44.0, average_reward: 35.5, length: 43
episode: 262, total reward: 16.0, average_reward: 35.2, length

episode: 371, total reward: 19.0, average_reward: 42.0, length: 18
episode: 372, total reward: 62.0, average_reward: 43.1, length: 61
episode: 373, total reward: 35.0, average_reward: 41.0, length: 34
episode: 374, total reward: 16.0, average_reward: 38.1, length: 15
episode: 375, total reward: 67.0, average_reward: 42.0, length: 66
episode: 376, total reward: 26.0, average_reward: 42.6, length: 25
episode: 377, total reward: 62.0, average_reward: 38.0, length: 61
episode: 378, total reward: 56.0, average_reward: 42.4, length: 55
episode: 379, total reward: 18.0, average_reward: 41.8, length: 17
episode: 380, total reward: 26.0, average_reward: 38.7, length: 25
episode: 381, total reward: 62.0, average_reward: 43.0, length: 61
episode: 382, total reward: 40.0, average_reward: 40.8, length: 39
episode: 383, total reward: 55.0, average_reward: 42.8, length: 54
episode: 384, total reward: 24.0, average_reward: 43.6, length: 23
episode: 385, total reward: 56.0, average_reward: 42.5, length

episode: 494, total reward: 61.0, average_reward: 66.1, length: 60
episode: 495, total reward: 51.0, average_reward: 62.8, length: 50
episode: 496, total reward: 57.0, average_reward: 65.3, length: 56
episode: 497, total reward: 136.0, average_reward: 74.8, length: 135
episode: 498, total reward: 22.0, average_reward: 72.2, length: 21
episode: 499, total reward: 42.0, average_reward: 65.2, length: 41
episode: 500, total reward: 79.0, average_reward: 69.8, length: 78
episode: 501, total reward: 51.0, average_reward: 63.3, length: 50
episode: 502, total reward: 38.0, average_reward: 65.4, length: 37
episode: 503, total reward: 33.0, average_reward: 57.0, length: 32
episode: 504, total reward: 83.0, average_reward: 59.2, length: 82
episode: 505, total reward: 39.0, average_reward: 58.0, length: 38
episode: 506, total reward: 32.0, average_reward: 55.5, length: 31
episode: 507, total reward: 43.0, average_reward: 46.2, length: 42
episode: 508, total reward: 49.0, average_reward: 48.9, leng

...

episode: 3524, total reward: 192.0, average_reward: 199.2, length: 191
episode: 3525, total reward: 200.0, average_reward: 199.2, length: 199
episode: 3526, total reward: 200.0, average_reward: 199.2, length: 199
episode: 3527, total reward: 200.0, average_reward: 199.2, length: 199
episode: 3528, total reward: 200.0, average_reward: 199.2, length: 199
episode: 3529, total reward: 200.0, average_reward: 199.2, length: 199
episode: 3530, total reward: 200.0, average_reward: 199.2, length: 199
episode: 3531, total reward: 200.0, average_reward: 199.2, length: 199
episode: 3532, total reward: 200.0, average_reward: 199.2, length: 199
episode: 3533, total reward: 200.0, average_reward: 199.2, length: 199
episode: 3534, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3535, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3536, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3537, total reward: 169.0, average_reward: 196.9, length: 168
episod

episode: 3640, total reward: 200.0, average_reward: 182.8, length: 199
episode: 3641, total reward: 200.0, average_reward: 182.8, length: 199
episode: 3642, total reward: 200.0, average_reward: 182.8, length: 199
episode: 3643, total reward: 200.0, average_reward: 182.8, length: 199
episode: 3644, total reward: 176.0, average_reward: 191.5, length: 175
episode: 3645, total reward: 200.0, average_reward: 197.6, length: 199
episode: 3646, total reward: 200.0, average_reward: 197.6, length: 199
episode: 3647, total reward: 200.0, average_reward: 197.6, length: 199
episode: 3648, total reward: 200.0, average_reward: 197.6, length: 199
episode: 3649, total reward: 200.0, average_reward: 197.6, length: 199
episode: 3650, total reward: 200.0, average_reward: 197.6, length: 199
episode: 3651, total reward: 200.0, average_reward: 197.6, length: 199
episode: 3652, total reward: 200.0, average_reward: 197.6, length: 199
episode: 3653, total reward: 179.0, average_reward: 195.5, length: 178
episod

episode: 3756, total reward: 200.0, average_reward: 191.2, length: 199
episode: 3757, total reward: 200.0, average_reward: 191.2, length: 199
episode: 3758, total reward: 200.0, average_reward: 199.8, length: 199
episode: 3759, total reward: 200.0, average_reward: 199.8, length: 199
episode: 3760, total reward: 200.0, average_reward: 199.8, length: 199
episode: 3761, total reward: 137.0, average_reward: 193.5, length: 136
episode: 3762, total reward: 200.0, average_reward: 193.5, length: 199
episode: 3763, total reward: 200.0, average_reward: 193.5, length: 199
episode: 3764, total reward: 200.0, average_reward: 193.7, length: 199
episode: 3765, total reward: 200.0, average_reward: 193.7, length: 199
episode: 3766, total reward: 200.0, average_reward: 193.7, length: 199
episode: 3767, total reward: 200.0, average_reward: 193.7, length: 199
episode: 3768, total reward: 200.0, average_reward: 193.7, length: 199
episode: 3769, total reward: 200.0, average_reward: 193.7, length: 199
episod

episode: 3872, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3873, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3874, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3875, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3876, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3877, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3878, total reward: 171.0, average_reward: 197.1, length: 170
episode: 3879, total reward: 200.0, average_reward: 197.1, length: 199
episode: 3880, total reward: 183.0, average_reward: 195.4, length: 182
episode: 3881, total reward: 167.0, average_reward: 192.1, length: 166
episode: 3882, total reward: 200.0, average_reward: 192.1, length: 199
episode: 3883, total reward: 200.0, average_reward: 192.1, length: 199
episode: 3884, total reward: 200.0, average_reward: 192.1, length: 199
episode: 3885, total reward: 200.0, average_reward: 192.1, length: 199
episod

episode: 3988, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3989, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3990, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3991, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3992, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3993, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3994, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3995, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3996, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3997, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3998, total reward: 200.0, average_reward: 200.0, length: 199
episode: 3999, total reward: 200.0, average_reward: 200.0, length: 199
episode: 4000, total reward: 200.0, average_reward: 200.0, length: 199
episode: 4001, total reward: 200.0, average_reward: 200.0, length: 199
episod